In [2]:
#For importing modules
import os, sys

#For file handling
from glob import glob
from os import listdir
from os.path import isfile, join
import json 

#For image processing
import PIL
import cv2
import nibabel as nib
import nibabel.processing 
from nibabel.processing import resample_to_output
import nilearn

#Image visualisation
import matplotlib.pyplot as plt

#Other
import numpy as np
import math
from random import sample

#Modeling
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from nilearn.image import concat_imgs, mean_img, resample_img

import tensorflow as tflow
from tensorflow.keras.layers import Flatten
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
import tensorflow as tf
import scipy
import sklearn
from sklearn.model_selection import train_test_split
from nilearn.image import concat_imgs, mean_img, resample_img

import tensorflow as tflow
from tensorflow.keras.layers import Flatten, Dense, Dropout
from keras.layers.core import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import os

import nibabel as nib
import nilearn

import json 

import numpy as np
import math
from random import sample

from sklearn.utils.class_weight import compute_class_weight

In [4]:
sys.path.append('/path/to/2014_07_13_test')
import generate_input

In [5]:
!ls './'

README.md                batch-load-preprocess.py test.ipynb
__pycache__              generate_input.py
baseline-model.ipynb     preprocessed-data


In [6]:
!ls './preprocessed-data/images/'

case_00000.nii.gz case_00075.nii.gz case_00150.nii.gz case_00225.nii.gz
case_00001.nii.gz case_00076.nii.gz case_00151.nii.gz case_00226.nii.gz
case_00002.nii.gz case_00077.nii.gz case_00152.nii.gz case_00227.nii.gz
case_00003.nii.gz case_00078.nii.gz case_00153.nii.gz case_00228.nii.gz
case_00004.nii.gz case_00079.nii.gz case_00154.nii.gz case_00229.nii.gz
case_00005.nii.gz case_00080.nii.gz case_00155.nii.gz case_00230.nii.gz
case_00006.nii.gz case_00081.nii.gz case_00156.nii.gz case_00231.nii.gz
case_00007.nii.gz case_00082.nii.gz case_00157.nii.gz case_00232.nii.gz
case_00008.nii.gz case_00083.nii.gz case_00158.nii.gz case_00233.nii.gz
case_00009.nii.gz case_00084.nii.gz case_00159.nii.gz case_00234.nii.gz
case_00010.nii.gz case_00085.nii.gz case_00160.nii.gz case_00235.nii.gz
case_00011.nii.gz case_00086.nii.gz case_00161.nii.gz case_00236.nii.gz
case_00012.nii.gz case_00087.nii.gz case_00162.nii.gz case_00237.nii.gz
case_00013.nii.gz case_00088.nii.gz case_00163.nii.gz case_00238

In [7]:
x_train, y_train, x_val, y_val, x_test, y_test = generate_input.generate_data_input(
    '../kits21/kits21/data/kits.json', n_samples = 50, neg_pct = 100, rootdir = './preprocessed-data/', 
    img_dir = 'images', mask_dir = 'masks', val_split = 10, test_split = 10)

Number of labels in labels_dict:  300
Generated list of 50 case IDs of 275 positive and 25 negative labels
Are image and mask path list the same lenght?:  True
Are image and mask dictionaries the same lenght?: True
Are cases and labels lists the same length?:  True 50 50
Generated training, validation and test with 30, 10 and 10 cases.
Are training set and labels lists the same length?:  True 6549 6549
After adding channels:  (6549, 224, 224, 3) (2748, 224, 224, 3) (1444, 224, 224, 3)


## upsampling 

In [8]:
#from imblearn.over_sampling import SMOTE

#x_train_reshape = x_train.reshape(x_train.shape[0], 224 * 224 * 1)
#sm = SMOTE(random_state=42)
#x_smote, y_smote = sm.fit_resample(x_train_reshape, y_train)
#x_smote = x_smote.reshape(x_smote.shape[0], 224, 224, 1)


## Developing baseline model

In [9]:
# draft model
restnet = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3), classes=1)

# metrics_list = [tf.keras.metrics.SpecificityAtSensitivity(0.7, name = 'specificity'),
#                tf.keras.metrics.AUC(name = 'auc'),
#                tf.keras.metrics.FalsePositives(name = 'fp'),
#                tf.keras.metrics.TrueNegatives(name = 'tn'),
#                 tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),]

#calculate class weights
#class_weights_array = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_train), y = np.concatenate(y_train, axis=0))
#class_weights = dict(zip(np.unique(y_train), class_weights_array ))


optimizer_fn = tf.keras.optimizers.experimental.RMSprop(learning_rate=0.00002, jit_compile = False)


model = Sequential()
model.add(restnet)
model.add(Dense(224, activation='relu', input_dim=(224, 224, 3)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizer_fn, loss='binary_crossentropy', metrics= ['accuracy'] )
model.summary()
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=3, batch_size = 32 )


Metal device set to: Apple M1


2023-03-24 16:24:15.969518: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-24 16:24:15.970888: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dense (Dense)               (None, 7, 7, 224)         458976    
                                                                 
 dropout (Dropout)           (None, 7, 7, 224)         0         
                                                                 
 flatten (Flatten)           (None, 10976)             0         
                                                                 
 dense_1 (Dense)             (None, 1)                 10977     
                                                                 
Total params: 24,057,665
Trainable params: 24,004,545
Non-trainable params: 53,120
_________________________________________________________________
Epoch 1/3


2023-03-24 16:24:23.822380: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-24 16:24:26.283818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


205/205 [==============================] - ETA: 0s - loss: 0.4497 - accuracy: 0.9606

2023-03-24 16:28:42.037864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


205/205 [==============================] - 291s 1s/step - loss: 0.4497 - accuracy: 0.9606 - val_loss: 1.8510 - val_accuracy: 0.3745
Epoch 2/3
205/205 [==============================] - 281s 1s/step - loss: 0.0248 - accuracy: 0.9939 - val_loss: 0.9080 - val_accuracy: 0.6175
Epoch 3/3
205/205 [==============================] - 283s 1s/step - loss: 0.0054 - accuracy: 0.9986 - val_loss: 0.9864 - val_accuracy: 0.6255


In [10]:
results = model.evaluate(x_val, y_val)
results

86/86 [==============================] - 32s 375ms/step - loss: 0.9864 - accuracy: 0.6255


[0.9863938689231873, 0.625545859336853]